# Goals

1) Aquire transaction data from a local game store for the years 2021-23 <br>
2) Prepare data for use in future projects <br>
3) Document preperation process <br>

In [1]:
# Imports

# libraries
import pandas as pd
import regex as re

import os

# writen files
import cata_lists.accessories as a
import cata_lists.board_games as b
import cata_lists.concessions as c
import cata_lists.paint_supplies as p
import cata_lists.rpg as r
import cata_lists.table_minis as m
import cata_lists.tcg as t
import cata_lists.other as o



# Read in Data and Create Dataframe

In [2]:
# combine annual csv files into one dataframe

df_2021 = pd.read_csv('raw_data/2021-2022.csv').sort_values('Date')
df_2022 = pd.read_csv('raw_data/2022-2023.csv').sort_values('Date')
df_2023 = pd.read_csv('raw_data/2023-2024.csv').sort_values('Date')


df = pd.concat([df_2021, df_2022, df_2023]).sort_values('Date')

In [3]:
df.head()

,Date,Time,Time Zone,Gross Sales,Discounts,Service Charges,Net Sales,Gift Card Sales,Tax,Tip,...,Deposit Details,Fee Percentage Rate,Fee Fixed Rate,Refund Reason,Discount Name,Transaction Status,Cash App,Order Reference ID,Fulfillment Note,Free Processing Applied
6119,2021-01-01,15:07:02,Central Time (US & Canada),$11.99,$0.00,$0.00,$11.99,$0.00,$0.99,$0.00,...,https://squareup.com/dashboard/sales/deposits/...,2.6,$0.10,NaN,NaN,Complete,$0.00,NaN,NaN,$0.00
6114,2021-01-01,19:52:02,Central Time (US & Canada),$76.96,-$7.70,$0.00,$69.26,$0.00,$5.71,$0.00,...,https://squareup.com/dashboard/sales/deposits/...,2.6,$0.10,NaN,Military,Complete,$0.00,NaN,NaN,$0.00
6115,2021-01-01,19:36:24,Central Time (US & Canada),$257.14,-$25.50,$0.00,$231.64,$0.00,$19.11,$0.00,...,https://squareup.com/dashboard/sales/deposits/...,2.6,$0.10,NaN,Military,Complete,$0.00,NaN,NaN,$0.00
6118,2021-01-01,15:09:20,Central Time (US & Canada),$5.99,$0.00,$0.00,$5.99,$0.00,$0.49,$0.00,...,https://squareup.com/dashboard/sales/deposits/...,2.6,$0.10,NaN,NaN,Complete,$0.00,NaN,NaN,$0.00
6117,2021-01-01,16:30:31,Central Time (US & Canada),$31.25,$0.00,$0.00,$31.25,$0.00,$2.58,$0.00,...,https://squareup.com/dashboard/sales/deposits/...,2.6,$0.10,NaN,NaN,Complete,$0.00,NaN,NaN,$0.00


In [4]:
df['Event Type'].value_counts()

Payment    19696
Refund        25
Name: Event Type, dtype: int64

In [5]:
df[df['Event Type'] == 'Refund'][['Net Sales', 'Description', 'Refund Reason', 'Customer ID']]

,Net Sales,Description,Refund Reason,Customer ID
4740,-$4.99,Opaque Dice (Regular),Returned Goods,NaN
4225,-$13.99,Zombie Dice (Regular) - SJG131313,Returned Goods,NaN
4114,-$59.99,Agricola (Regular),Returned Goods,NaN
4027,-$100.00,Mystery MTG Box $100 (Regular),Accidental Charge,NaN
929,-$49.99,Sherlock Holmes Consulting Detective: Jack The...,Accidental Charge,NaN
605,$0.00,"Big Red (Regular), Coke (Regular), Cardfight V...",Accidental Charge,NaN
5549,-$81.46,16 x Cardfight Vanguard: Awakening Of Chakraba...,Returned Goods,NaN
4659,-$61.35,"Custom Amount, Custom Amount",Accidental Charge,NaN
3875,-$152.74,30 x Kamigawa Set Booster (Regular),Accidental Charge,NaN
7149,-$9.50,Dominaria Remastered- Collector Booster (Regular),Accidental Charge,NaN


In [6]:
df[df['Event Type'] == 'Refund'][['Net Sales', 'Description', 'Refund Reason', 'Customer ID']].shape

(25, 4)

Merged dataframe contains:
* 19,721 rows, each representing a transaction occuring between 2021 and 2023
* 51 columns providing details about those transactions

# Drop Irrelevant Colunms
Future projects will focus on finding patterns in customer purchasing behavior, so I will restrict the data frame only to coluns that are relevant to that purpose

In [7]:
# remake df with only relevant columns

df = df[['Date',
         'Time',
         'Gross Sales',
         'Discounts',
         'Net Sales',
         'Transaction ID',
         'Customer ID', 
         'Description', 
         'Discount Name',
         'Event Type']]

In [8]:
df.head()

,Date,Time,Gross Sales,Discounts,Net Sales,Transaction ID,Customer ID,Description,Discount Name,Event Type
6119,2021-01-01,15:07:02,$11.99,$0.00,$11.99,kG5juYfQ9JD0GvtEkoi5pMfeV,T49C25V8WS37VB4RSJTBN13TSR,Dragon Shield Sleeves: Matte Blue (Regular),NaN,Payment
6114,2021-01-01,19:52:02,$76.96,-$7.70,$69.26,I0xk8oW1vCFdvw6R5NKnmL6eV,9K5STWJGVD61B0FGW6CQ08MQQW,"Jumpstart Booster Pack (Regular), Dragon Shiel...",Military,Payment
6115,2021-01-01,19:36:24,$257.14,-$25.50,$231.64,4MdmaO7EAiG9KtMf2E5rR6yeV,F4RYR1ARCD7ZDD4E9PFYRVF6H0,"Dex Binder (Regular), 3 x Custom Amount, Candy...",Military,Payment
6118,2021-01-01,15:09:20,$5.99,$0.00,$5.99,s0C9IZpNhTIrAxb2cwwPbHyeV,F36VZJMBMH3PH5SN664GPV4NQR,Ultra Pro-100 Deck Box White 2020 (Regular) - ...,NaN,Payment
6117,2021-01-01,16:30:31,$31.25,$0.00,$31.25,ajAARRigYcH0BI0l8LCYOeAfV,BBBRQPD57S3YV4GJG1CCB6ATGC,"Candy (Regular), Dungeons & Dragons: Icewind D...",NaN,Payment


# Rename Columns
Columns were renamed for clarity ease of use

In [9]:
df = df.rename(columns = {'Date' : 'date',
                          'Time' : 'time',
                          'Gross Sales' : 'gross_sales',
                          'Discounts': 'discount_amount',
                          'Net Sales' : 'net_sales',
                          'Transaction ID' : 'trans_id',
                          'Customer ID' : 'cust_id', 
                          'Description' : 'cart', 
                          'Discount Name' : 'discount_type',
                          'Event Type' : 'event_type'})

df.columns

Index(['date', 'time', 'gross_sales', 'discount_amount', 'net_sales',
       'trans_id', 'cust_id', 'cart', 'discount_type', 'event_type'],
      dtype='object')

After dropping and renaming data contains the following columns
* date
* time
* gross_sales
* discount_amount
* net_sales
* trans_id, id number for transaction
* cust_id, id number for customer making purchase
* cart, string containing names of items bought
* discount_type, type of discount applied to purchase
* event_type, type of transaction

# Handle Null Values, and Check Column Data Types
<br>
* 3445 nulls in cust_id were imputed with 'unknown'<br>
* 10236 nulls in discount_type were imputed with 'No Discount'<br>
* 3 rows were dropped that contained null values in cart and no other useful data
* Cleaned columns containing dollor amounts and converted them to float

In [10]:
# handle null values
df.cust_id = df.cust_id.fillna('unknown')

df.discount_type = df.discount_type.fillna('no_discount')

df = df.dropna(subset=['cart'])

# convert dollors from string to float
df['net_sales'] = df['net_sales'].str.replace('$', '').str.replace(',', '').astype(float)
df['gross_sales'] = df['gross_sales'].str.replace('$', '').str.replace(',', '').astype(float)
df['discount_amount'] = df['discount_amount'].str.replace('$', '').str.replace(',', '').astype(float)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()


# Set Datetime as Index and add Time Derivative Columns

In [11]:
# create datetime column and set it as the index
df['datetime'] = df.date + ' ' + df.time
df['datetime'] = pd.to_datetime(df['datetime'])

df = df.set_index('datetime').sort_index()

df = df.drop(columns = ['date', 'time'])

# get time derivative columns
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['weekday'] = df.index.day_name()  

In [12]:
df.head()

,gross_sales,discount_amount,net_sales,trans_id,cust_id,cart,discount_type,event_type,year,month,day,weekday
datetime,,,,,,,,,,,,
2021-01-01 15:07:02,11.99,0.0,11.99,kG5juYfQ9JD0GvtEkoi5pMfeV,T49C25V8WS37VB4RSJTBN13TSR,Dragon Shield Sleeves: Matte Blue (Regular),no_discount,Payment,2021,1,1,Friday
2021-01-01 15:09:20,5.99,0.0,5.99,s0C9IZpNhTIrAxb2cwwPbHyeV,F36VZJMBMH3PH5SN664GPV4NQR,Ultra Pro-100 Deck Box White 2020 (Regular) - ...,no_discount,Payment,2021,1,1,Friday
2021-01-01 16:30:31,31.25,0.0,31.25,ajAARRigYcH0BI0l8LCYOeAfV,BBBRQPD57S3YV4GJG1CCB6ATGC,"Candy (Regular), Dungeons & Dragons: Icewind D...",no_discount,Payment,2021,1,1,Friday
2021-01-01 16:31:27,21.25,0.0,21.25,oOUNX1T69j9NK1Cc8W1dLyseV,W67T53BZFD5VQ8J3W7387X8SW0,"Candy (Regular), Double Sided Battlemap (Regul...",no_discount,Payment,2021,1,1,Friday
2021-01-01 19:36:24,257.14,-25.5,231.64,4MdmaO7EAiG9KtMf2E5rR6yeV,F4RYR1ARCD7ZDD4E9PFYRVF6H0,"Dex Binder (Regular), 3 x Custom Amount, Candy...",Military,Payment,2021,1,1,Friday


In [13]:
df.head()

,gross_sales,discount_amount,net_sales,trans_id,cust_id,cart,discount_type,event_type,year,month,day,weekday
datetime,,,,,,,,,,,,
2021-01-01 15:07:02,11.99,0.0,11.99,kG5juYfQ9JD0GvtEkoi5pMfeV,T49C25V8WS37VB4RSJTBN13TSR,Dragon Shield Sleeves: Matte Blue (Regular),no_discount,Payment,2021,1,1,Friday
2021-01-01 15:09:20,5.99,0.0,5.99,s0C9IZpNhTIrAxb2cwwPbHyeV,F36VZJMBMH3PH5SN664GPV4NQR,Ultra Pro-100 Deck Box White 2020 (Regular) - ...,no_discount,Payment,2021,1,1,Friday
2021-01-01 16:30:31,31.25,0.0,31.25,ajAARRigYcH0BI0l8LCYOeAfV,BBBRQPD57S3YV4GJG1CCB6ATGC,"Candy (Regular), Dungeons & Dragons: Icewind D...",no_discount,Payment,2021,1,1,Friday
2021-01-01 16:31:27,21.25,0.0,21.25,oOUNX1T69j9NK1Cc8W1dLyseV,W67T53BZFD5VQ8J3W7387X8SW0,"Candy (Regular), Double Sided Battlemap (Regul...",no_discount,Payment,2021,1,1,Friday
2021-01-01 19:36:24,257.14,-25.5,231.64,4MdmaO7EAiG9KtMf2E5rR6yeV,F4RYR1ARCD7ZDD4E9PFYRVF6H0,"Dex Binder (Regular), 3 x Custom Amount, Candy...",Military,Payment,2021,1,1,Friday


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19718 entries, 2021-01-01 15:07:02 to 2023-12-31 18:59:48
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   gross_sales      19718 non-null  float64
 1   discount_amount  19718 non-null  float64
 2   net_sales        19718 non-null  float64
 3   trans_id         19718 non-null  object 
 4   cust_id          19718 non-null  object 
 5   cart             19718 non-null  object 
 6   discount_type    19718 non-null  object 
 7   event_type       19718 non-null  object 
 8   year             19718 non-null  int64  
 9   month            19718 non-null  int64  
 10  day              19718 non-null  int64  
 11  weekday          19718 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 2.0+ MB


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19718 entries, 2021-01-01 15:07:02 to 2023-12-31 18:59:48
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   gross_sales      19718 non-null  float64
 1   discount_amount  19718 non-null  float64
 2   net_sales        19718 non-null  float64
 3   trans_id         19718 non-null  object 
 4   cust_id          19718 non-null  object 
 5   cart             19718 non-null  object 
 6   discount_type    19718 non-null  object 
 7   event_type       19718 non-null  object 
 8   year             19718 non-null  int64  
 9   month            19718 non-null  int64  
 10  day              19718 non-null  int64  
 11  weekday          19718 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 2.0+ MB


In [16]:
print(len(df))

df = df.dropna(subset=['cart'])

print(len(df))

19718
19718


In [17]:
df.cust_id.fillna('unknown')


datetime
2021-01-01 15:07:02    T49C25V8WS37VB4RSJTBN13TSR
2021-01-01 15:09:20    F36VZJMBMH3PH5SN664GPV4NQR
2021-01-01 16:30:31    BBBRQPD57S3YV4GJG1CCB6ATGC
2021-01-01 16:31:27    W67T53BZFD5VQ8J3W7387X8SW0
2021-01-01 19:36:24    F4RYR1ARCD7ZDD4E9PFYRVF6H0
                                  ...            
2023-12-31 16:06:59    9QF5CQDWN165DD0QQTJDESHQN0
2023-12-31 17:07:49                       unknown
2023-12-31 17:11:52    8V1DCCMP0104HDHCTCJRKZ7HTG
2023-12-31 18:41:21    4FH2YZRSWS1P7F2Y9CP65AA37R
2023-12-31 18:59:48    TVCHWKSK351RZAZT7K3W04VXZ0
Name: cust_id, Length: 19718, dtype: object